In [34]:
import numpy as np
import numba as nb
import math as mt
import copy
import timeit
from numba import prange

In [20]:
N = 10
K = 1000
M = 10
A = np.arange(N*K).astype(np.float64)
A = A.reshape(N,K)
B = np.arange(K*M).astype(np.float64)
B = B.reshape(K,M)
BT = np.ascontiguousarray(B.T)
print(A.shape)
print(B.shape)
print(BT.shape)

(10, 1000)
(1000, 10)
(10, 1000)


In [21]:
def product(A,B):
    N = A.shape[0]
    K = A.shape[1]
    M = B.shape[1]
    #print(N, M, K)
    C = np.empty(shape=[N,M])
    for i in range(N):
        for j in range(M):
            buf = 0
            for k in range(K):
                buf = buf + A[i][k] * B[k][j]
            C[i][j] = buf
    return C

def productT(A,B):
    N = A.shape[0]
    K = A.shape[1]
    M = B.shape[0]
    C = np.empty(shape=[N,M])
    for i in range(N):
        for j in range(M):
            buf = 0
            for k in range(K):
                buf = buf + A[i][k] * B[j][k]
            C[i][j] = buf
    return C


In [23]:
%timeit product(A,B)
%timeit productT(A,BT)

83.4 ms ± 4.2 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
73.8 ms ± 550 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [24]:
@nb.njit(nb.float64[:,::1](nb.float64[:,::1],nb.float64[:,::1]))
def NBproduct(A,B):
    N = A.shape[0]
    K = A.shape[1]
    M = B.shape[1]
    C = np.empty(shape=(N,M), dtype=np.float64)
    for i in range(N):
        for j in range(M):
            buf = 0
            for k in range(K):
                buf = buf + A[i][k] * B[k][j]
            C[i][j] = buf
    return C

@nb.njit(nb.float64[:,::1](nb.float64[:,::1],nb.float64[:,::1]))
def NBproductT(A,B):
    N = A.shape[0]
    K = A.shape[1]
    M = B.shape[0]
    C = np.empty(shape=(N,M), dtype=np.float64)
    for i in range(N):
        for j in range(M):
            buf = 0
            for k in range(K):
                buf = buf + A[i][k] * B[j][k]
            C[i][j] = buf
    return C

%timeit NBproduct(A,B)
%timeit NBproductT(A,BT)

111 µs ± 8.1 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
99.9 µs ± 159 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [25]:
%timeit A@B

8.08 µs ± 550 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [29]:
def productIJK(A,B):
    N = A.shape[0]
    K = A.shape[1]
    M = B.shape[1]
    C = np.empty(shape=[N,M])
    for i in range(N):
        for j in range(M):
            buf = 0
            for k in range(K):
                buf = buf + A[i][k] * B[k][j]
            C[i][j] = buf
    return C

def productJIK(A,B):
    N = A.shape[0]
    K = A.shape[1]
    M = B.shape[1]
    C = np.empty(shape=[N,M])
    for j in range(M):
        for i in range(N):
            buf = 0
            for k in range(K):
                buf = buf + A[i][k] * B[k][j]
            C[i][j] = buf
    return C

def productIKJ(A,B):
    N = A.shape[0]
    K = A.shape[1]
    M = B.shape[1]
    C = np.empty(shape=[N,M])
    for i in range(N):
        for k in range(K):
            buf = 0
            for j in range(M):
                buf = buf + A[i][k] * B[k][j]
            C[i][j] = buf
    return C

def productJKI(A,B):
    N = A.shape[0]
    K = A.shape[1]
    M = B.shape[1]
    C = np.empty(shape=[N,M])
    for j in range(M):
        for k in range(K):
            buf = 0
            for i in range(N):
                buf = buf + A[i][k] * B[k][j]
            C[i][j] = buf
    return C

def productKIJ(A,B):
    N = A.shape[0]
    K = A.shape[1]
    M = B.shape[1]
    C = np.empty(shape=[N,M])
    for k in range(K):
        for i in range(N):
            buf = 0
            for j in range(M):
                buf = buf + A[i][k] * B[k][j]
            C[i][j] = buf
    return C

def productKJI(A,B):
    N = A.shape[0]
    K = A.shape[1]
    M = B.shape[1]
    C = np.empty(shape=[N,M])
    for k in range(K):
        for j in range(M):
            buf = 0
            for i in range(N):
                buf = buf + A[i][k] * B[k][j]
            C[i][j] = buf
    return C

In [30]:
%timeit productIJK(A,B)
%timeit productJIK(A,B)
%timeit productIKJ(A,B)
%timeit productJKI(A,B)
%timeit productKIJ(A,B)
%timeit productKJI(A,B)

79.6 ms ± 4.13 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
74.6 ms ± 403 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
87 ms ± 4.37 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
91.9 ms ± 6.16 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
98.5 ms ± 6.01 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
94.2 ms ± 7.22 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [38]:
@nb.njit(nb.float64[:,::1](nb.float64[:,::1],nb.float64[:,::1]))
def NproductIJK(A,B):
    N = A.shape[0]
    K = A.shape[1]
    M = B.shape[1]
    C = np.empty(shape=(N,M), dtype=np.float64)
    for i in prange(N):
        for j in prange(M):
            buf = 0
            for k in prange(K):
                buf = buf + A[i][k] * B[k][j]
            C[i][j] = buf
    return C

@nb.njit(nb.float64[:,::1](nb.float64[:,::1],nb.float64[:,::1]))
def NproductJIK(A,B):
    N = A.shape[0]
    K = A.shape[1]
    M = B.shape[1]
    C = np.empty(shape=(N,M), dtype=np.float64)
    for j in prange(M):
        for i in prange(N):
            buf = 0
            for k in prange(K):
                buf = buf + A[i][k] * B[k][j]
            C[i][j] = buf
    return C

@nb.njit(nb.float64[:,::1](nb.float64[:,::1],nb.float64[:,::1]))
def NproductIKJ(A,B):
    N = A.shape[0]
    K = A.shape[1]
    M = B.shape[1]
    C = np.empty(shape=(N,M), dtype=np.float64)
    for i in prange(N):
        for k in prange(K):
            buf = 0
            for j in prange(M):
                buf = buf + A[i][k] * B[k][j]
            C[i][j] = buf
    return C

@nb.njit(nb.float64[:,::1](nb.float64[:,::1],nb.float64[:,::1]))
def NproductJKI(A,B):
    N = A.shape[0]
    K = A.shape[1]
    M = B.shape[1]
    C = np.empty(shape=(N,M), dtype=np.float64)
    for j in prange(M):
        for k in prange(K):
            buf = 0
            for i in prange(N):
                buf = buf + A[i][k] * B[k][j]
            C[i][j] = buf
    return C

@nb.njit(nb.float64[:,::1](nb.float64[:,::1],nb.float64[:,::1]))
def NproductKIJ(A,B):
    N = A.shape[0]
    K = A.shape[1]
    M = B.shape[1]
    C = np.empty(shape=(N,M), dtype=np.float64)
    for k in prange(K):
        for i in prange(N):
            buf = 0
            for j in prange(M):
                buf = buf + A[i][k] * B[k][j]
            C[i][j] = buf
    return C

@nb.njit(nb.float64[:,::1](nb.float64[:,::1],nb.float64[:,::1]))
def NproductKJI(A,B):
    N = A.shape[0]
    K = A.shape[1]
    M = B.shape[1]
    C = np.empty(shape=(N,M), dtype=np.float64)
    for k in prange(K):
        for j in prange(M):
            buf = 0
            for i in prange(N):
                buf = buf + A[i][k] * B[k][j]
            C[i][j] = buf
    return C

In [39]:
%timeit NproductIJK(A,B)
%timeit NproductJIK(A,B)
%timeit NproductIKJ(A,B)
%timeit NproductJKI(A,B)
%timeit NproductKIJ(A,B)
%timeit NproductKJI(A,B)

105 µs ± 3.47 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
101 µs ± 948 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
44.8 µs ± 95.9 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
62 µs ± 4.24 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
59.8 µs ± 289 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
51.5 µs ± 103 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [40]:
def Strassen(A,B):
    if (A.shape[0] != A.shape[1]) & (B.shape[0] != B.shape[1]) & (A.shape[0] != B.shape[0]):
        print("Not a square matrix")
        return
    N = A.shape[0]
    if N>=64:
        return A@B
    p = mt.ceil(mt.log2(N))
    dim = 2**p
    A2 = A
    B2 = B
    delta = dim - N
    for i in range(delta):
        adh = np.zeros((N,1))
        A2 = np.hstack((A2, adh))
        B2 = np.hstack((B2, adh))
    for i in range(delta):
        adv = np.zeros((1,dim))
        A2 = np.vstack((A2,adv))
        B2 = np.vstack((B2,adv))
    half = (2**(p-1))
    A11 = A2[:half,:half]
    A12 = A2[:half,half:]
    A21 = A2[half:,:half]
    A22 = A2[half:,half:]
    B11 = B2[:half,:half]
    B12 = B2[:half,half:]
    B21 = B2[half:,:half]
    B22 = B2[half:,half:]
    P1 = Strassen((A11 + A22), (B11 + B22))
    P2 = Strassen((A21 + A22), B11)
    P3 = Strassen(A11, (B12 - B22))
    P4 = Strassen(A22, (B21 - B11))
    P5 = Strassen((A11 + A12), B22)
    P6 = Strassen((A21 - A11), (B11 + B12))
    P7 = Strassen((A12 - A22), (B21 + B22))
    C11 = P1 + P4 - P5 + P7
    C12 = P3 + P5
    C21 = P2 + P4
    C22 = P1 - P2 + P3 + P6
    C1 = np.hstack((C11,C12))
    C2 = np.hstack((C21,C22))
    C = np.vstack((C1,C2))
    C = C[:N,:N]
    return C

In [41]:
N = 100
C = np.random.sample((N,N))
D = np.random.sample((N,N))

In [42]:
%timeit Strassen(D,D)
%timeit C@D

29.9 µs ± 3.43 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
26.2 µs ± 998 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
